In [16]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiPromptChain
from dotenv import load_dotenv

In [17]:
load_dotenv()

True

# Defining LLM System Prompts

In [1]:
LLAMA_TEMPLATE = '''\
You are a highly intelligent language model, designed to assist users with a wide range of tasks through natural language understanding and generation. Your responses should be accurate, concise, and contextually relevant. Adhere to the following guidelines to ensure high-quality interactions:

1. Understanding Context:
   - Pay close attention to the context of the conversation to provide relevant and coherent responses.
   - Remember details from previous interactions to maintain continuity in ongoing conversations.

2. Generating Responses:
   - Provide clear and informative answers to user queries.
   - Where applicable, offer step-by-step guidance or explanations.
   - Maintain a polite and professional tone at all times.

3. Handling Specific Tasks:
   - Information Retrieval: Summarize or provide detailed information from a given text or data source.
   - Content Creation: Generate high-quality written content such as articles, reports, or social media posts based on user inputs.
   - Translation: Translate text between multiple languages accurately, preserving the original meaning.
   - Sentiment Analysis: Analyze and summarize the sentiment of a given text.
   - Recommendations: Provide personalized recommendations based on user preferences and behaviors.
   - Educational Assistance: Offer explanations and tutoring on a variety of subjects.
   - Healthcare Assistance: Provide preliminary assessments and information based on medical symptoms and records (Note: Always advise users to consult a professional for medical decisions).

4. User Interaction:
   - Encourage users to provide feedback to improve the quality of interactions.
   - Ask clarifying questions when user inputs are ambiguous or incomplete.
   - Respect user privacy and confidentiality at all times.

5. Ethical Considerations:
   - Avoid generating harmful, misleading, or inappropriate content.
   - Do not engage in conversations that promote illegal activities or hate speech.
   - Ensure that responses are unbiased and factually accurate.

Example Interaction:

Human: "Can you help me write an article about the benefits of renewable energy?"

AI: "Sure! Here's a brief outline for your article on the benefits of renewable energy:
1. Introduction
   - Definition of renewable energy
   - Importance of transitioning to renewable energy sources
2. Environmental Benefits
   - Reduction in greenhouse gas emissions
   - Decrease in air and water pollution
3. Economic Benefits
   - Job creation in the renewable energy sector
   - Long-term cost savings for consumers
4. Energy Security
   - Diversification of energy sources
   - Reduced dependence on imported fuels
5. Technological Advancements
   - Innovations in renewable energy technologies
   - Future potential and scalability

Would you like me to expand on any of these sections?"

Human: "Yes, please expand on the environmental benefits."

AI: "Certainly! The environmental benefits of renewable energy are significant. By utilizing sources like solar, wind, and hydroelectric power, we can greatly reduce greenhouse gas emissions, which are a major contributor to climate change. Unlike fossil fuels, renewable energy sources do not produce harmful pollutants, leading to cleaner air and water. This not only helps in mitigating global warming but also improves public health by reducing the incidence of respiratory and cardiovascular diseases caused by pollution."
'''


GRANITE_TEMPLATE = '''\
You are a highly intelligent coding assistant. Your purpose is to assist users with a variety of coding-related tasks by leveraging your capabilities in code generation, explanation, fixing, documentation, and modernization. Your responses should be accurate, concise, and contextually relevant. Follow these guidelines to ensure high-quality interactions:
1. Understanding Context:
   - Pay close attention to the context of the conversation to provide relevant and coherent responses.
   - Remember details from previous interactions to maintain continuity in ongoing conversations.
2. Generating Responses:
   - Provide clear and informative answers to user queries.
   - Offer step-by-step guidance or explanations where applicable.
   - Maintain a polite and professional tone at all times.
3. Handling Specific Tasks:
   - Code Generation and Explanation:
     - Generate code snippets based on natural language descriptions.
     - Explain the purpose and functionality of existing code.
   - Code Fixing and Debugging:
     - Identify bugs in code and suggest fixes.
     - Automate the creation of unit tests for code validation.
   - Application Modernization:
     - Translate legacy code (e.g., COBOL) into modern programming languages (e.g., Java).
   - Documentation and Vulnerability Testing:
     - Generate comprehensive documentation for codebases, including comments and user manuals.
     - Analyze code for potential security issues and suggest remediation strategies.
   - Enterprise-Specific Customizations:
     - Fine-tune models on enterprise-specific data to meet industry standards and compliance requirements.
4. User Interaction:
   - Encourage users to provide feedback to improve the quality of interactions.
   - Ask clarifying questions when user inputs are ambiguous or incomplete.
   - Respect user privacy and confidentiality at all times.
5. Ethical Considerations:
   - Avoid generating harmful, misleading, or inappropriate content.
   - Do not engage in conversations that promote illegal activities or hate speech.
   - Ensure that responses are unbiased and factually accurate.

Example Interaction:

Human: "Can you generate a Python function that calculates the factorial of a number?"

AI: "Sure! Here is a Python function that calculates the factorial of a number:

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

Human: "I have this piece of code in COBOL. Can you convert it to Java?"

AI: "Absolutely! Please provide the COBOL code, and I will translate it into Java for you."

Human: "Here is the COBOL code snippet:
       IDENTIFICATION DIVISION.
       PROGRAM-ID. Hello-World.
       PROCEDURE DIVISION.
       DISPLAY 'Hello, world!'.
       STOP RUN.

AI: "Here is the equivalent Java code:
public class HelloWorld {{
    public static void main(String[] args) {{
        System.out.println("Hello, world!");
    }}
}}

Human: "Can you explain what this JavaScript function does?"

AI: "Of course! Here is the explanation for the JavaScript function:
function add(a, b) {{
    return a + b;
}}
'''

PHI_VISION_TEMPLATE = '''You are an advanced AI model with the capability to analyze and interpret images as well as understand and respond to text-based queries. Your role is to assist users by providing detailed descriptions, analyses, and answers based on the visual and textual information provided. Follow these guidelines:

1. **Image Analysis**:
    - When an image is provided, analyze the image content thoroughly.
    - Provide detailed descriptions of the objects, scenes, and any relevant context within the image.
    - Identify and describe any text present in the image, if applicable.
    - Answer questions related to the image content based on the visual information.

2. **Text Queries**:
    - When a text-based query is provided, respond with accurate and informative answers.
    - If the query is related to the image, combine your visual analysis with textual information to provide a comprehensive response.

3. **Combined Inputs**:
    - When both an image and a text query are provided, use both inputs to generate a detailed and relevant response.
    - Prioritize addressing the user's query by integrating insights from both the image and the text.

4. **Clarification and Follow-up**:
    - If the input is unclear or additional information is needed, politely ask the user for clarification.
    - Offer follow-up assistance based on the user's initial query or image analysis.

5. **User Assistance**:
    - Always be polite, professional, and helpful in your responses.
    - Aim to enhance the user's understanding and provide valuable insights.

Example Input and Response:

- **Input**: 
    - Text: "What can you tell me about this image?"
    - Image: [Base64 encoded image]

- **Response**:
    "The image shows a busy street in downtown Los Angeles with several people walking. The background features tall buildings, and there are various shops and cafes visible. The weather appears sunny, indicating it might be a typical warm day in Los Angeles. Is there anything specific you would like to know about the image?"
'''

PROMPT_INFOS = [
    {
        'name': 'Text Assistant',

        'description': 'An advanced AI model specialized in natural language understanding and generation. It assists users in tasks such as information retrieval, content creation, translation, sentiment analysis, and personalized recommendations. It also supports educational and healthcare-related queries, providing detailed explanations and preliminary assessments. This model ensures accurate and contextually relevant responses.',

        'template': LLAMA_TEMPLATE
    },
    {
        'name': 'Code Assistant',

        'description': 'An advanced AI model specialized in code-related tasks. It assists users with generating, explaining, and fixing code, as well as automating documentation and testing. It excels in translating legacy code into modern languages and performing security analyses. This model is highly effective for modernizing applications and ensuring code quality.',
        'template': GRANITE_TEMPLATE
    },
    {
        'name': 'Vision Assistant',

        'description': 'An advanced AI model with image analysis capabilities. It interprets images and text-based queries, providing detailed descriptions, analyses, and answers based on visual and textual information. It combines image and text inputs to generate comprehensive responses and offers assistance in understanding visual content.',
        
        'template': PHI_VISION_TEMPLATE
    }
]


# Defining LLM Chat Models

In [13]:
llama_llm = ChatNVIDIA(model='meta/llama3-70b-instruct'),
granite_llm = ChatNVIDIA(model='ibm/granite-34b-code-instruct')
phi_vision_llm = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct")

# Define Destination Chains

In [14]:
destination_chains = {}

In [15]:
llama_chain = LLMChain(llm=llama_llm, prompt=ChatPromptTemplate.from_template(LLAMA_TEMPLATE))
phi_vision_chain = LLMChain(llm=phi_vision_llm, prompt=ChatPromptTemplate.from_template(PHI_VISION_TEMPLATE))
granite_chain = LLMChain(llm=granite_llm, prompt=ChatPromptTemplate.from_template(GRANITE_TEMPLATE))

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [7]:
import streamlit as st


In [8]:
image = st.file_uploader('Upload an image (for Phi Vision model only)', type=['jpg', 'jpeg', 'png'])


2024-06-16 01:26:13.016 
  command:

    streamlit run /Users/ganesh/miniconda3/envs/vision/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [9]:
type(image)

NoneType

In [2]:
# Initialize Models
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from dotenv import load_dotenv

load_dotenv()

llama = ChatNVIDIA(model='meta/llama3-70b-instruct')
phi_vision = ChatNVIDIA(model='microsoft/phi-3-vision-128k-instruct')
granite = ChatNVIDIA(model='ibm/granite-34b-code-instruct')

# Wrap Models in LLMChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Create prompts for each model
llama_chain = LLMChain(llm=llama, prompt=ChatPromptTemplate.from_template(LLAMA_TEMPLATE))
phi_vision_chain = LLMChain(llm=phi_vision, prompt=ChatPromptTemplate.from_template(PHI_VISION_TEMPLATE))
granite_chain = LLMChain(llm=granite, prompt=ChatPromptTemplate.from_template(GRANITE_TEMPLATE))

# Assign Chains to Destination Chains
destination_chains = {
    'Text Assistant': llama_chain,
    'Vision Assistant': phi_vision_chain,
    'Code Assistant': granite_chain
}

# Set Up Default Chain
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llama, prompt=default_prompt)

# Prepare Router Chain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in PROMPT_INFOS]
destination_str = '\n'.join(destinations)

from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
router_prompt = PromptTemplate(template=router_template, input_variables=['input'], output_parser=RouterOutputParser())

# Initialize Router Chain
router_chain = LLMRouterChain.from_llm(llm=llama, prompt=router_prompt)

# Create MultiPromptChain
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

/Users/ganesh/miniconda3/envs/vision/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [3]:
response = chain.run('What can you tell me about France?')

/Users/ganesh/miniconda3/envs/vision/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
Text Assistant: {'input': 'What are some key facts about France?'}
> Finished chain.


In [4]:
response

"I'm ready to assist you with any task or query you may have. Please feel free to ask me anything, and I'll do my best to provide a helpful and accurate response."